Something about the Workflow of this notebook


First need to select features from timeseries

In [1]:
%run ../bin/features/ts_2_featuresets.py

/home/ec2-user/pgv_ml/psuedosyn_gnssvel/data/synth_ts/212_05.pq
There are 16 CPUs on this machine 


Then translate ambient data set to features

In [ ]:
%run ../bin/features/ambient_2_fs.py

In [1]:
%run ../bin/features/jgr_fs_2_new.py

There are 16 CPUs on this machine 
Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.


Now train model using nested xval /kfold grid search optimization

(best run as nohup)

In [ ]:
%run ../bin/models/nest_xval_synth_MP.py

In [ ]:
pq_list

In [1]:
from itertools import product
from collections import deque

import os
import pandas as pd
import glob
import numpy as np

import datetime
from scipy import signal
import pywt

import pyarrow as pa
import pyarrow.parquet as pq
import json

import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score

import sys
sys.path.insert(0, os.path.abspath('../bin/models'))

from nested_xval_utils import *

from multiprocessing import Pool, cpu_count

In [2]:
pq_list=[os.path.join(os.path.dirname(os.getcwd()), 'data/feature_sets/',f) for f in os.listdir(os.path.join(os.path.dirname(os.getcwd()), 'data/feature_sets/'))]
#pd_list=[pd.read_parquet(pq) for pq in pq_list if ".pq" in pq]
meta_list=[read_meta(pq_fs) for pq_fs in pq_list if ".pq" in pq_fs]
meta_df=pd.DataFrame.from_records(meta_list)

##########
fs={'feature':['all','psd', 'wavelet', 'time'], 'stacking':['horizontal'], 'dims':[['H0','H1','UP']], 'augment':[True, False]}
fs={'feature':['psd'], 'stacking':['horizontal'], 'dims':[['H0','H1','UP']], 'augment':[True]}
feature_sets=[dict(zip(fs, v)) for v in product(*fs.values())]
d = {'n_folds':[5],'max_depth': [100], 'n_estimators': [100], 'class_wt':[None], 'wl_thresh':[-15,15],} #'wl_thresh':[-15, -10, -5, 0]
d = {'n_folds':[5],'max_depth': [10], 'n_estimators': [10], 'class_wt':[None], 'wl_thresh':[15],} #'wl_thresh':[-15, -10, -5, 0]
hyperp=[dict(zip(d, v)) for v in product(*d.values())]
#########

results=[]
outer_results =[]
first_tp=True
first_fp=True



#convert to rsn
#test_set=meta_df[meta_df.eq_name.isin(test_set_events)].record_number.unique()
#train_set=meta_df[meta_df.eq_name.isin(train_set_events)].record_number.unique()


In [10]:
ambient_list= list(meta_df[meta_df.magnitude.isnull()].eq_name.unique())
event_list=meta_df[~meta_df.magnitude.isnull()].sort_values(['magnitude'], ascending=False).groupby("eq_name").count().sort_values(['station'], ascending=False).index.tolist()
full_list=ambient_list+event_list

In [11]:
# number of waveforms
print(len(meta_df[~meta_df.magnitude.isnull()].record_number.unique()))
# of events
print(len(event_list))

1970
167


In [8]:
######################
test_set_list=[]
train_set_list=[]
run_list=[]
num_runs=10
for k in np.arange(num_runs):
#for k in np.arange(1):
    run=k+1
    items = deque(full_list)
    items.rotate(-k)
    test_set_events=list(items)[::num_runs]
    train_set_events=list(set(full_list) - set(test_set_events))
    test_set_list.append(test_set_events)
    train_set_list.append(train_set_events)
    run_list.append(run)

In [10]:
test_set_list

[['ambient_6',
  'ambient_5',
  'ambient_23',
  'ambient_25',
  'ambient_46',
  ' "Northridge-01"',
  ' "Coalinga-01"',
  ' "Imperial Valley-06"',
  ' "Kobe_ Japan"',
  ' "Coalinga-05"',
  ' "Hector Mine"',
  ' "Mammoth Lakes-03"',
  ' "Gilroy"',
  ' "Lazio Abruzzo (aftershock 1)_ Italy"',
  ' "Umbria Marche (aftershock 3)_ Italy"',
  ' "Sicilia-Orientale_ Italy"',
  ' "Northwest China-04"',
  ' "Central Calif-02"',
  ' "Imperial Valley-05"',
  ' "Kern County"',
  ' "Northwest China-01"',
  ' "Drama_ Greece"'],
 ['ambient_0',
  'ambient_2',
  'ambient_20',
  'ambient_35',
  'ambient_47',
  ' "Whittier Narrows-01"',
  ' "Whittier Narrows-02"',
  ' "14383980"',
  ' "Chi-Chi_ Taiwan-04"',
  ' "Coalinga-04"',
  ' "Livermore-01"',
  ' "Parkfield"',
  ' "Georgia_ USSR"',
  ' "Superstition Hills-02"',
  ' "Umbria-03_ Italy"',
  ' "Coalinga-08"',
  ' "Umbria Marche (aftershock 15)_ Italy"',
  ' "Spitak_ Armenia"',
  ' "Northern Calif-07"',
  ' "Managua_ Nicaragua-02"',
  ' "Northern Calif-03"'

In [ ]:
test_set_list=[]
train_set_list=[]
run_list=[]
num_runs=10
num_runs=1
#for k in np.arange(num_runs):
for k in np.arange(num_runs):
    run=k+1
    items = deque(full_list)
    items.rotate(-k)
    test_set_events=list(items)[::num_runs]
    train_set_events=list(set(full_list) - set(test_set_events))
    test_set_list.append(test_set_events)
    train_set_list.append(train_set_events)
    run_list.append(run)

In [ ]:
full_list

1970

In [ ]:
meta_df.dtypes

In [ ]:
str_cols=['station','year','eq_name','record_number','mechanism','noise_lev']

meta_df[str_cols]

In [ ]:
meta_df[str_cols] = meta_df[str_cols].astype('|S') 

In [ ]:
meta_df

In [ ]:
meta_df.to_csv('../data/meta/fs_meta.csv', index=False)

In [ ]:
meta_df2=pd.read_csv('../data/meta/fs_meta.csv')

In [ ]:
meta_df2

In [ ]:
meta_df2=meta_df2['station'].astype('str')

In [ ]:
meta_df2.dtypes